In [1]:
%load_ext lab_black

In [2]:
import json
import httpx
import string
import secrets

from time import sleep
from pydantic import BaseModel

from getpass import getpass

In [3]:
class DeploymentContext(BaseModel):
    env: dict = {}

In [4]:
# base_url = "http://localhost:8001"
base_url = "https://deploy.staging.wersdoerfer.de"
username = "jochen"
password = getpass()
service = "single_cast_hosting"

# get access_token
client = httpx.Client(base_url=base_url)
r = client.post("/token", data={"username": username, "password": password})
access_token = r.json()["access_token"]
access_token_headers = {"authorization": f"Bearer {access_token}"}

# get service token
data = {"service": service, "origin": "notebook"}
r = client.post("/service-token", json=data, headers=access_token_headers)
service_token = r.json()["service_token"]
headers = {"authorization": f"Bearer {service_token}"}
client.headers = headers

 ····················


# Deploy

In [5]:
def show_details(details_url):
    r = client.get(details_url, headers=headers)
    rdata = r.json()
    len_steps = len([s for s in rdata["steps"] if s["state"] == "success"])
    started = rdata["started"]
    finished = rdata["finished"]
    print(f"{len_steps} {started} {finished}")

## Create Service

In [6]:
alphabet = string.ascii_letters + string.digits
database_password = "".join(secrets.choice(alphabet) for i in range(20))
secret_key = "".join(secrets.choice(alphabet) for i in range(32))
fqdn = "happypodcast.staging.django-cast.com"
underscored_fqdn = fqdn.replace(".", "_")
site_id = f"cast_{underscored_fqdn}"
env = {
    "fqdn": fqdn,
    "site_id": site_id,
    "user_name": "cast_1",
    "database_name": site_id,
    "database_user": site_id,
    "database_password": database_password,
    "secret_key": secret_key,
    "port": 10001,
    "settings_file_name": site_id,
}
env

{'fqdn': 'happypodcast.staging.django-cast.com',
 'site_id': 'cast_happypodcast_staging_django-cast_com',
 'user_name': 'cast_1',
 'database_name': 'cast_happypodcast_staging_django-cast_com',
 'database_user': 'cast_happypodcast_staging_django-cast_com',
 'database_password': 'vlmiT4oZo12CeVYoykxV',
 'secret_key': 'IWWR6DC4iTW4vHyucrfZpeplUzOlcBZ1',
 'port': 10001,
 'settings_file_name': 'cast_happypodcast_staging_django-cast_com'}

### Get Service Token

In [7]:
data = {"service": service, "origin": "notebook"}
r = client.post("/service-token", json=data, headers=access_token_headers)
service_token = r.json()["service_token"]
headers = {"authorization": f"Bearer {service_token}"}

In [14]:
context = DeploymentContext(env=env)
r = client.post("/deployments/", json=context.dict(), headers=headers)
rdata = r.json()
print(rdata)
details_url = r.json()["details"]

# for i in range(3):
#     show_details(details_url)
#     sleep(1)

{'id': 223, 'service_id': 66, 'origin': 'notebook', 'user': 'jochen', 'started': '2022-07-02T07:33:52.320450+00:00', 'finished': None, 'context': {'env': {'fqdn': 'happypodcast.staging.django-cast.com', 'site_id': 'cast_happypodcast_staging_django-cast_com', 'user_name': 'cast_1', 'database_name': 'cast_happypodcast_staging_django-cast_com', 'database_user': 'cast_happypodcast_staging_django-cast_com', 'database_password': 'vlmiT4oZo12CeVYoykxV', 'secret_key': 'IWWR6DC4iTW4vHyucrfZpeplUzOlcBZ1', 'port': 10001, 'settings_file_name': 'cast_happypodcast_staging_django-cast_com'}}, 'details': 'https://deploy.staging.wersdoerfer.de/deployments/223'}


In [12]:
json.dumps(context.env)

'{"fqdn": "happypodcast.staging.django-cast.com", "site_id": "cast_happypodcast_staging_django-cast_com", "user_name": "cast_1", "database_name": "cast_happypodcast_staging_django-cast_com", "database_user": "cast_happypodcast_staging_django-cast_com", "database_password": "vlmiT4oZo12CeVYoykxV", "secret_key": "IWWR6DC4iTW4vHyucrfZpeplUzOlcBZ1", "port": 10001, "settings_file_name": "cast_happypodcast_staging_django-cast_com"}'

## Remove Service

### Get Service Token

In [19]:
data = {"service": "single_cast_hosting_remove", "origin": "notebook"}
r = client.post("/service-token", json=data, headers=access_token_headers)
service_token = r.json()["service_token"]
headers = {"authorization": f"Bearer {service_token}"}

In [20]:
context = DeploymentContext(env=env)
r = client.post("/deployments/", json=context.dict(), headers=headers)
rdata = r.json()
print(rdata)
details_url = r.json()["details"]

{'id': 224, 'service_id': 67, 'origin': 'notebook', 'user': 'jochen', 'started': '2022-07-02T07:36:21.794363+00:00', 'finished': None, 'context': {'env': {'fqdn': 'happypodcast.staging.django-cast.com', 'site_id': 'cast_happypodcast_staging_django-cast_com', 'user_name': 'cast_1', 'database_name': 'cast_happypodcast_staging_django-cast_com', 'database_user': 'cast_happypodcast_staging_django-cast_com', 'database_password': 'vlmiT4oZo12CeVYoykxV', 'secret_key': 'IWWR6DC4iTW4vHyucrfZpeplUzOlcBZ1', 'port': 10001, 'settings_file_name': 'cast_happypodcast_staging_django-cast_com'}}, 'details': 'https://deploy.staging.wersdoerfer.de/deployments/224'}


In [21]:
context.env

{'fqdn': 'happypodcast.staging.django-cast.com',
 'site_id': 'cast_happypodcast_staging_django-cast_com',
 'user_name': 'cast_1',
 'database_name': 'cast_happypodcast_staging_django-cast_com',
 'database_user': 'cast_happypodcast_staging_django-cast_com',
 'database_password': 'vlmiT4oZo12CeVYoykxV',
 'secret_key': 'IWWR6DC4iTW4vHyucrfZpeplUzOlcBZ1',
 'port': 10001,
 'settings_file_name': 'cast_happypodcast_staging_django-cast_com'}